# Q* Learning with FrozenLake 🕹️⛄
<br> 
In this Notebook, we'll implement an agent <b>that plays FrozenLake.</b>

The goal of this game is <b>to go from the starting state (S) to the goal state (G)</b> by walking only on frozen tiles (F) and avoid holes (H).However, the ice is slippery, <b>so you won't always move in the direction you intend (stochastic environment)</b>

## Step 0: Import the dependencies 📚
We use 3 libraries:
- `Numpy` for our Qtable
- `OpenAI Gym` for our FrozenLake Environment
- `Random` to generate random numbers

In [1]:
import numpy as np
import gym
import random

## Step 1: Create the environment 🎮
- Here we'll create the FrozenLake environment. 
- OpenAI Gym is a library <b> composed of many environments that we can use to train our agents.</b>
- In our case we choose to use Frozen Lake.

In [2]:
env = gym.make("FrozenLake-v1",map_name="8x8")

## Step 2: Create the Q-table and initialize it 🗄️
- Now, we'll create our Q-table, to know how much rows (states) and columns (actions) we need, we need to calculate the action_size and the state_size
- OpenAI Gym provides us a way to do that: `env.action_space.n` and `env.observation_space.n`

In [11]:
action_size = env.action_space.n
state_size = env.observation_space.n
print(action_size)
print(state_size)

4
64


In [4]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Step 3: Create the hyperparameters ⚙️
- Here, we'll specify the hyperparameters

In [5]:
total_episodes = 30000        # Total episodes
learning_rate = 0.65           # Learning rate
max_steps = 500                # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.0001             # Exponential decay rate for exploration prob

## Step 4: The Q learning algorithm 🧠
- Now we implement the Q learning algorithm:

In [6]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()[0]
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, truncated,info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state,:]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

Score over time: 0.09296666666666667
[[2.40007266e-02 2.45566769e-02 2.70421162e-02 2.43356508e-02]
 [2.67993538e-02 2.57520620e-02 3.78499254e-02 2.39843764e-02]
 [3.77013875e-02 4.50211968e-02 3.03247866e-02 3.26324545e-02]
 [4.56456736e-02 5.61240465e-02 7.20767484e-02 7.53861256e-02]
 [6.17735774e-02 8.26676656e-02 9.01708129e-02 6.27107144e-02]
 [6.73966901e-02 6.88013370e-02 8.78291386e-02 6.84243765e-02]
 [8.88374885e-02 7.65150403e-02 9.35738770e-02 8.41434158e-02]
 [9.23461974e-02 8.77996792e-02 9.24829509e-02 9.19456679e-02]
 [2.36441274e-02 2.38664467e-02 2.45226533e-02 2.36904531e-02]
 [2.40823470e-02 2.50907718e-02 2.94612729e-02 2.49435839e-02]
 [2.62139600e-02 2.31437183e-02 6.17614408e-02 3.94759047e-02]
 [1.96075469e-03 3.62459419e-03 2.23706305e-03 9.32086042e-02]
 [3.94891102e-02 6.68040305e-02 5.56376801e-02 1.02064681e-01]
 [5.37298286e-02 7.63140647e-02 9.16369083e-02 7.53450283e-02]
 [9.80846796e-02 9.40749993e-02 1.20574144e-01 9.55179014e-02]
 [9.15402614e-02 9

## Step 5: Use our Q-table to play FrozenLake ! 👾

In [7]:
env.reset()

result = 0
for episode in range(1000):
    state = env.reset()[0]
    step = 0
    done = False
    #print("****************************************************")
    #print("EPISODE ", episode+1)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, truncated, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            result += reward
            # We print the number of step it took.
            #print("Number of steps", step)
            #print("Reward",reward)
            break
        state = new_state

print("Completionrate",result/1000)
env.close()

C:\Users\fromm\Anaconda3\lib\site-packages\gym\envs\toy_text\frozen_lake.py:271: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("FrozenLake-v1", render_mode="rgb_array")
  logger.warn(


Completionrate 0.669


<h1>Run Game with rendering</h1>

In [10]:
env = gym.make("FrozenLake-v1",render_mode="human",map_name="8x8")
env.reset()

result = 0
for episode in range(5):
    state = env.reset()[0]
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode+1)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, truncated, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            result += reward
            # We print the number of step it took.
            print("Number of steps", step)
            print("Reward",reward)
            break
        state = new_state

print("Completionrate",result/5)
env.close()

****************************************************
EPISODE  1
Number of steps 57
Reward 1.0
****************************************************
EPISODE  2
Number of steps 54
Reward 0.0
****************************************************
EPISODE  3
Number of steps 20
Reward 0.0
****************************************************
EPISODE  4
Number of steps 127
Reward 0.0
****************************************************
EPISODE  5
Number of steps 169
Reward 1.0
Completionrate 0.4
